## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:
![](./fig/h4q1.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea： 
### 在大模型帮助下完成
1. 证明：假设存在$T_B$是BST，但是不是MST。则$T_B$的总权重比MST $T_M$更大。但是$T_B$的最大权重边更小，这与MST定义矛盾，因为如果$T_B$的最大权重边比$T_M$的更小，那么$T_M$的总权重不可能更小。因此每个MST一定是BST
2. 想法如下
* 要求算法的时间复杂度是线性，因此不能使用传统的 MST 算法,如 Kruskal 或 Prim，因为它们的时间复杂度通常涉及排序或优先队列操作，无法满足线性复杂度的要求。
* 只考虑权重不超过 b 的边，因为任何超过 b 的边都不可能被包含在 BST 中,并查集的操作可以满足线性时间复杂度的要求,可以使用并查集数据结构来快速判断这些边是否足以形成一个连通的生成树
* 如果图中存在一个生成树，并且所有边的权重都不超过 b，那么并查集将能够成功连接所有节点；反之，如果图中不存在这样的生成树，那么并查集将无法连接所有节点。
3. 想法如下：
首先将图中的边按权重从小到大排序。然后选择边构建生成树，从小到大选择边，如果这条边的两个顶点不在同一个连通分量中，就将他们加入生成树，继续这个过程直到生成树包含V-1条边。
最后找到最大权重边，生成树中最大权重边就是瓶颈值。
因为还是要求线性时间复杂度因此还是需要使用并查集。



In [2]:
# add your code here
class UnionFind:
    def __init__(self, size):
        # 初始化父节点和秩（树的高度上界）
        self.parent = list(range(size))
        self.rank = [0] * size

    def find(self, x):
        # 查找 x 的根节点，路径压缩
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        # 合并 x 和 y 所在的集合，按秩合并
        x_root = self.find(x)
        y_root = self.find(y)
        if x_root == y_root:
            return False  # 已经在同一个集合中
        if self.rank[x_root] < self.rank[y_root]:
            self.parent[x_root] = y_root
        else:
            self.parent[y_root] = x_root
            if self.rank[x_root] == self.rank[y_root]:
                self.rank[x_root] += 1
        return True

def has_bounded_bottleneck_tree(edges, b, n):
    # 判断是否存在最大权重边不超过 b 的生成树
    # edges: 边列表，每个边为元组 (u, v, w)
    # b: 权重上限
    # n: 顶点数

    # 过滤出权重不超过 b 的边
    valid_edges = [e for e in edges if e[2] <= b]

    uf = UnionFind(n)
    
    # 尝试用这些边连接所有顶点
    for u, v, w in valid_edges:
        uf.union(u, v)  # 尝试合并两个顶点所在的集合

    # 检查所有顶点是否连通
    root = uf.find(0)  # 找到第一个顶点的根
    for i in range(1, n):
        if uf.find(i) != root:  # 如果某个顶点的根不同，则不连通
            return False
    return True
# algorithm of the liear time complexity   # O(E) E为边的数量


In [3]:
class UnionFind:
    # 并查集类，用于高效管理元素的分组情况
    def __init__(self, size):
        # 初始化父节点和秩（树的高度上界）
        self.parent = list(range(size))
        self.rank = [0] * size

    def find(self, x):
        # 查找 x 的根节点，路径压缩
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        # 合并 x 和 y 所在的集合，按秩合并
        x_root = self.find(x)
        y_root = self.find(y)
        if x_root == y_root:
            return False  # 已经在同一个集合中
        if self.rank[x_root] < self.rank[y_root]:
            self.parent[x_root] = y_root
        else:
            self.parent[y_root] = x_root
            if self.rank[x_root] == self.rank[y_root]:
                self.rank[x_root] += 1
        return True

def find_bottleneck_tree(edges, n):
    # 找到图的瓶颈生成树
    # edges: 边列表，每个边为元组 (u, v, w)
    # n: 顶点数

    # 对边按权重从小到大排序
    sorted_edges = sorted(edges, key=lambda x: x[2])

    uf = UnionFind(n)
    selected_edges = []
    max_weight = 0

    # 遍历排序后的边，构建生成树
    for u, v, w in sorted_edges:
        if uf.find(u) != uf.find(v):  # 如果两个顶点不在同一个集合中
            uf.union(u, v)  # 合并这两个集合
            selected_edges.append((u, v, w))  # 将边加入生成树
            max_weight = max(max_weight, w)  # 更新最大权重

            # 如果生成树已经包含 V-1 条边，提前退出
            if len(selected_edges) == n - 1:
                break

    # 检查是否所有顶点都连通
    root = uf.find(0)
    for i in range(1, n):
        if uf.find(i) != root:
            return None, None  # 图不连通，无法形成生成树

    return selected_edges, max_weight


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


idea：
### 该题在大模型帮助下完成
预先计算出所有城市对的最短路径，然后对于每一条新边 (u,v)，通过比较来计算其潜在的改进。
1. 计算所有顶点对的初始最短路径，通过多次使用 Dijkstra 算法来实现
2. 计算每条可能的新边对距离的改进:对于每条新边 (u,v)，我们检查所有顶点对 (s,t)，并计算新的最短路径是否比原来的更短。
3. 选择能够使某个顶点对 (s,t) 的最短路径减少最多的新边。

In [4]:
import heapq

def dijkstra(graph, start, n):
    # 使用 Dijkstra 算法计算从 start 到其他所有顶点的最短路径
    # graph: 邻接表表示的图
    # start: 起始顶点
    # n: 顶点数
    dist = [float('inf')] * n  # 初始化距离数组为无穷大
    dist[start] = 0  # 起始顶点到自身的距离为 0
    heap = [(0, start)]  # 使用优先队列（最小堆），存储（距离，顶点）
    
    while heap:
        current_dist, u = heapq.heappop(heap)  # 提取具有最小距离的顶点
        # 如果当前距离大于已知的最短距离，跳过
        if current_dist > dist[u]:
            continue
        # 遍历顶点 u 的所有邻接顶点
        for v, weight in graph[u]:
            # 如果发现更短的路径，更新距离并加入优先队列
            if dist[v] > dist[u] + weight:
                dist[v] = dist[u] + weight
                heapq.heappush(heap, (dist[v], v))
    return dist

def find_best_new_road(original_graph, new_roads, n):
    # 计算原始图中所有顶点对的最短路径
    # original_graph: 原始图的邻接表
    # new_roads: 可能添加的新边列表
    # n: 顶点数
    # 预计算原始图中所有顶点对的最短路径
    original_shortest = [dijkstra(original_graph, i, n) for i in range(n)]
    
    max_improvement = 0  # 记录最大改进
    best_road = None  # 记录最优新边
    best_s = None  # 记录得到最大改进的起始城市
    best_t = None  # 记录得到最大改进的目标城市
    
    # 检查每条新边对所有顶点对的影响
    for u, v, new_length in new_roads:
        # 创建一个新的图，包含新边 (u, v)
        new_graph = [original_graph[i].copy() for i in range(n)]
        new_graph[u].append((v, new_length))
        
        # 计算新图中所有顶点对的最短路径
        new_shortest = [dijkstra(new_graph, i, n) for i in range(n)]
        
        # 比较原始路径和新路径，寻找最大改进
        for s in range(n):
            for t in range(n):
                if s != t:
                    improvement = original_shortest[s][t] - new_shortest[s][t]
                    # 更新最大改进和对应的新边及顶点对
                    if improvement > max_improvement:
                        max_improvement = improvement
                        best_road = (u, v, new_length)
                        best_s = s
                        best_t = t
    
    # 输出结果
    if best_road:
        print(f"添加边 {best_road} 可以使城市 {best_s} 到 {best_t} 的距离减少 {max_improvement}")
    else:
        print("没有新边可以减少任何城市对的距离")


正确性分析：
1. 算法考虑了所有可能的新边和所有顶点对，确保不会遗漏任何潜在的改进
2. 使用 Dijkstra 算法保证了最短路径计算的准确性。
3. 通过遍历所有可能性并记录最大改进，确保找到全局最优解。

复杂度分析：
1. 预处理最短路径复杂度：每次 Dijkstra 的时间复杂度为 O(E+VlogV)，总复杂度为 O(V⋅(E+VlogV))。
2. 评估每条新边复杂性：对于每条新边，重新运行 Dijkstra 算法的时间复杂度为 O(V⋅(E+VlogV))
总复杂度为 O(K⋅V⋅(E+VlogV))，其中 K 是新边的数量。

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：
### 本题在大模型帮助下完成
问题1： 
1. 顶点分割：将每个顶点 v $\in$ V 分割为两个顶点 $v_in$ 和 $v_out$
2. 添加边连接分割后的顶点：在$v_in$ 和 $v_out$之间添加一条有向边，其容量为顶点v的容量
3. 重新连接原有边:所有进入 v 的边现在进入 $v_in$ ,所有从 v 出发的边现在从 $v_out$出发
这样转换后的网络 G' 将具有性质： 每个顶点的容量被转换为一条边的容量。  边的数量和顶点的数量都线性增长，即 V′=O(V) 和 E′=O(E)。
在原始网络中，流经顶点 v 的流量不能超过其容量。在转换后的网络中，流经$v_in$到$v_out$的边的流量同样受到该容量的限制。因此，两个网络的最大流是相同的。

问题2：通过最大流算法来解决逃离问题
1. 构建流网络: 将网格中的每个顶点 (i,j) 视为一个节点,对于每个内部节点 (i,j)，连接其与四个邻居的边，容量为 1。对于边界节点，将其连接到一个超级汇点 t，边的容量为 1。
2. 设置源点和汇点: 添加一个超级源点 s，连接 s 到所有起点 ($x_k$,$y_k$)，边的容量为 1。边界节点连接到超级汇点 t，边的容量为 1。
3. 计算最大流：使用最大流算法计算从超级源点 s 到超级汇点 t 的最大流。
4. 判断是否存在逃离路径：如果最大流等于 m，则存在 m 条不相交的路径从起点到边界。否则，不存在这样的路径。
复杂度分析：
Dinic's 算法在稠密图中的时间复杂度为 O(E⋅V2),对于 n×n 网格，复杂度为 O(n^4)。



In [5]:
from collections import deque

# 定义边类
class Edge:
    def __init__(self, to, rev, capacity):
        # 目标节点
        self.to = to
        # 反向边的索引
        self.rev = rev
        # 边的容量
        self.capacity = capacity

# Dinic's 算法类
class Dinic:
    def __init__(self, n):
        # 初始化图的大小
        self.size = n
        # 邻接表表示图
        self.graph = [[] for _ in range(n)]
    
    # 添加边到图中
    def add_edge(self, fr, to, cap):
        # 正向边和反向边
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        # 添加正向边和反向边
        self.graph[fr].append(forward)
        self.graph[to].append(backward)
    
    # 使用 BFS 构建层次图
    def bfs_level(self, s, t, level):
        # 初始化队列
        q = deque()
        # 初始化层次数组
        level[:] = [-1] * self.size
        # 源点层次为0
        level[s] = 0
        q.append(s)
        # BFS遍历
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] < 0:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    # 如果到达汇点，退出
                    if edge.to == t:
                        return
    
    # 使用 DFS 在层次图中寻找阻塞流
    def dfs_flow(self, v, t, upTo, iter_, level):
        # 如果到达汇点，返回流量
        if v == t:
            return upTo
        # 遍历所有边
        for i in range(iter_[v], len(self.graph[v])):
            edge = self.graph[v][i]
            # 如果容量大于0且层次递增
            if edge.capacity > 0 and level[v] < level[edge.to]:
                # 递归寻找增广路径
                d = self.dfs_flow(edge.to, t, min(upTo, edge.capacity), iter_, level)
                if d > 0:
                    # 更新残留容量
                    edge.capacity -= d
                    self.graph[edge.to][edge.rev].capacity += d
                    return d
            # 更新迭代器
            iter_[v] += 1
        return 0
    
    # 计算最大流
    def max_flow(self, s, t):
        flow = 0
        level = [-1] * self.size
        while True:
            # 构建层次图
            self.bfs_level(s, t, level)
            # 如果汇点不可达，退出
            if level[t] < 0:
                return flow
            # 初始化迭代器
            iter_ = [0] * self.size
            while True:
                # 寻找阻塞流
                f = self.dfs_flow(s, t, float('inf'), iter_, level)
                if f <= 0:
                    break
                flow += f
            # 重置层次
            level = [-1] * self.size
        return flow

def escape_problem(grid_size, starts):
    # 构建图模型
    n = grid_size
    # 总节点数：网格节点 + 超级源点和汇点
    total_nodes = n * n + 2
    source = n * n  # 超级源点
    sink = n * n + 1  # 超级汇点
    dinic = Dinic(total_nodes)
    
    # 连接超级源点到所有起点
    for x, y in starts:
        node = x * n + y
        dinic.add_edge(source, node, 1)
    
    # 构建网格图
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # 四个方向
    for i in range(n):
        for j in range(n):
            node = i * n + j
            # 连接边界节点到超级汇点
            if i == 0 or i == n-1 or j == 0 or j == n-1:
                dinic.add_edge(node, sink, 1)
            # 连接相邻节点
            for di, dj in directions:
                ni, nj = i + di, j + dj
                if 0 <= ni < n and 0 <= nj < n:
                    dinic.add_edge(node, ni * n + nj, 1)
    
    # 计算最大流
    max_flow_val = dinic.max_flow(source, sink)
    # 判断是否所有起点都能逃离
    return max_flow_val == len(starts)

# 示例使用
n = 4
starts = [(0, 0), (1, 1), (2, 2)]
# 判断是否可以逃离
print(escape_problem(n, starts))

True
